In [3]:
#!git clone https://github.com/SparklePh/Seagate_Workshop

Cloning into 'Seagate_Workshop'...
remote: Enumerating objects: 2708, done.
remote: Total 2708 (delta 0), reused 0 (delta 0), pack-reused 2708 (from 2)
Receiving objects: 100% (2708/2708), 221.68 MiB | 31.85 MiB/s, done.
Resolving deltas: 100% (26/26), done.
Updating files: 100% (6117/6117), done.


In [6]:
# 1. ติดตั้ง Library (สำหรับ Colab ต้องรันบรรทัดนี้)
#!pip install line-bot-sdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 818.9/818.9 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 12.2 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# ส่งข้อความเข้า line

In [ ]:
import os
from flask import Flask, request, abort
from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    PushMessageRequest,
    TextMessage
)
from linebot.v3.webhooks import MessageEvent, TextMessageContent, JoinEvent

# --- 2. ดึงข้อมูลจาก Secrets ของ Colab ---
ACCESS_TOKEN = userdata.get('LINE_ACCESS_TOKEN')
USER_ID = userdata.get('MY_USER_ID')
GROUP_ID = userdata.get('GROUP_ID')

# --- 3. สร้าง Configuration ---


configuration = Configuration(access_token=ACCESS_TOKEN)


def send_hello():
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)

        # สร้างเนื้อหาข้อความ
        content = TextMessage(text="สวัสดี! ข้อความนี้ส่งมาจาก Python Messaging API ครับ 🚀")

        # คำสั่งส่ง (Push)
        request = PushMessageRequest(
            to=USER_ID,
            messages=[content]
        )

        line_bot_api.push_message(request)
        print("ส่งเรียบร้อย!")

if __name__ == "__main__":
    send_hello()

<class 'str'>
ส่งเรียบร้อย!


# ฝากรูปผ่าน web

In [4]:
import requests
import base64

def upload_to_imgbb(image_path, api_key):
    with open(image_path, "rb") as file:
        # แปลงไฟล์ภาพเป็น Base64
        payload = {
            "key": api_key,
            "image": base64.b64encode(file.read()),
        }
        # ส่งไปที่ API ของ Imgbb
        response = requests.post("https://api.imgbb.com/1/upload", payload)

        if response.status_code == 200:
            data = response.json()
            # ดึง Direct Link มาใช้งาน
            return data['data']['url']
        else:
            return f"Error: {response.text}"

# --- วิธีใช้งาน ---
MY_API_KEY = "0179f7de99a49feb3ee1a266ad27bdb4"
IMG_PATH = "/content/Seagate_Workshop/data/Tennis_Ball.jpg"

image_url = upload_to_imgbb(IMG_PATH, MY_API_KEY)
print(f"Direct Link สำหรับส่ง LINE: {image_url}")

Direct Link สำหรับส่ง LINE: https://i.ibb.co/YByXjrPF/c158727cb04c.jpg


# ส่งข้อความ + ส่งรูป

In [7]:
from google.colab import userdata
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    PushMessageRequest,
    ImageMessage,
    TextMessage
)

# --- 1. ดึง Key จาก Colab Secrets ---
# ต้องตั้งชื่อในรูปกุญแจให้ตรงกันนะครับ
ACCESS_TOKEN = userdata.get('LINE_ACCESS_TOKEN')
USER_ID = userdata.get('MY_USER_ID')
GROUP_ID = userdata.get('GROUP_ID')
MY_API_KEY = userdata.get('MY_API_KEY')

# --- 2. สร้าง Configuration ---
configuration = Configuration(access_token=ACCESS_TOKEN)

def send_image_only():
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)

        # 2. ใส่ File ID

        IMAGE_URL = "https://i.ibb.co/YByXjrPF/c158727cb04c.jpg"


        # 3. สร้าง ImageMessage
        # หมายเหตุ: LINE ต้องการทั้งรูปจริง (Original) และรูปพรีวิว (Preview)
        # ในที่นี้เราใช้ URL เดียวกันได้ครับ
        image_message = ImageMessage(
            original_content_url=IMAGE_URL,
            preview_image_url=IMAGE_URL
        )

        # เพิ่มข้อความปิดท้าย (Optional)
        text_message = TextMessage(text="ส่งรูปภาพสำเร็จ! 🎉")

        # 4. สร้าง Request และสั่งส่ง
        request = PushMessageRequest(
            to=USER_ID,
            messages=[image_message, text_message]
        )

        try:
            line_bot_api.push_message(request)
            print("✅ ส่งรูปภาพเรียบร้อยแล้ว!")
        except Exception as e:
            print(f"❌ เกิดข้อผิดพลาด: {e}")

if __name__ == "__main__":
    send_image_only()

✅ ส่งรูปภาพเรียบร้อยแล้ว!
